In [1]:
import rioxarray
import geopandas as gpd
import xarray as xr
from xrspatial.zonal import stats
from shapely.geometry import box
from rasterio.features import rasterize

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
layer = 'filldepth' # TODO make a list of raster layers to run (like the Control Table csv)
in_zone_data_path = '../high_res_data/NHDPLUS_H_1710_HU4_GDB.gdb'
output_path = '../high_res_data/output'
vpu_id = '1710' #TODO get all vpu_ids from national release GDB
raster_path = f'../high_res_data/NHDPLUS_H_{vpu_id}_HU4_RASTERS/HRNHDPlusRasters{vpu_id}/{layer}.tif'

In [3]:
raster = rioxarray.open_rasterio(raster_path).sel(band=1).drop_vars('band') #, chunks=True, lock=False
raster

<xarray.DataArray (y: 73988, x: 30913)> Size: 9GB
[2287191044 values with dtype=int32]
Coordinates:
  * x            (x) float64 247kB -2.306e+06 -2.306e+06 ... -1.997e+06
  * y            (y) float64 592kB 3.151e+06 3.151e+06 ... 2.411e+06 2.411e+06
    spatial_ref  int32 4B 0
Attributes: (12/14)
    AREA_OR_POINT:           Area
    DataType:                Generic
    RepresentationType:      THEMATIC
    STATISTICS_COVARIANCES:  1390409441.40587
    STATISTICS_MAXIMUM:      1084972
    STATISTICS_MEAN:         2167.2871929276
    ...                      ...
    STATISTICS_SKIPFACTORY:  1
    STATISTICS_STDDEV:       37288.194397233
    _FillValue:              -2147483648
    scale_factor:            1.0
    add_offset:              0.0
    units:                   centimetre

In [4]:
vector_data = gpd.read_file(in_zone_data_path, layer='NHDPlusCatchment')
vector_data.head()

,NHDPlusID,SourceFC,GridCode,AreaSqKm,VPUID,SHAPE_Length,SHAPE_Area,geometry
0,5.500010e+13,NHDPlusBurnLineEvent,587654,0.0389,1710,0.013786,0.000005,"MULTIPOLYGON (((-124.62939 48.21301, -124.6299..."
1,5.500010e+13,NHDPlusBurnLineEvent,26064,0.2719,1710,0.055632,0.000033,"MULTIPOLYGON (((-124.61768 48.21549, -124.6178..."
2,5.500010e+13,NHDPlusBurnLineEvent,3899,0.0507,1710,0.015380,0.000006,"MULTIPOLYGON (((-124.57229 48.217, -124.57253 ..."
3,5.500010e+13,NHDPlusBurnLineEvent,584956,0.0403,1710,0.011203,0.000005,"MULTIPOLYGON (((-124.52669 48.22661, -124.5267..."
4,5.500010e+13,NHDPlusBurnLineEvent,22600,0.4650,1710,0.039445,0.000056,"MULTIPOLYGON (((-124.67436 48.19532, -124.6744..."


In [5]:
vector_data = vector_data.to_crs(raster.rio.crs)
bounds = raster.rio.bounds()
bbox = box(*bounds)

vector_data_clipped = vector_data[vector_data.geometry.intersects(bbox)]

transform = raster.rio.transform()
width = raster.sizes['x']
height = raster.sizes['y']
# print(len(vector_data_clipped.GridCode.unique()))
shapes = [(geom, value) for geom, value in zip(vector_data_clipped.geometry, vector_data_clipped['GridCode'])]
shapes[1:10]

[(<MULTIPOLYGON (((-2113895 3122625, -2113905 3122625, -2113905 3122635, -2113...>,
  26064),
 (<MULTIPOLYGON (((-2110595 3121775, -2110595 3121835, -2110615 3121835, -2110...>,
  3899),
 (<MULTIPOLYGON (((-2107015 3121775, -2107015 3121785, -2107025 3121785, -2107...>,
  584956),
 (<MULTIPOLYGON (((-2118615 3121765, -2118625 3121765, -2118625 3121795, -2118...>,
  22600),
 (<MULTIPOLYGON (((-2108525 3121765, -2108525 3121775, -2108535 3121775, -2108...>,
  4080),
 (<MULTIPOLYGON (((-2116245 3121755, -2116265 3121755, -2116265 3121765, -2116...>,
  784030),
 (<MULTIPOLYGON (((-2106425 3121755, -2106435 3121755, -2106435 3121765, -2106...>,
  529615),
 (<MULTIPOLYGON (((-2109455 3124195, -2109465 3124195, -2109465 3124205, -2109...>,
  20315),
 (<MULTIPOLYGON (((-2108465 3124195, -2108495 3124195, -2108495 3124205, -2108...>,
  15377)]

In [6]:
zones = rasterize(
        shapes=shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0,  # Fill value for areas outside the vector data
        dtype='int32'
    )
zones

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [7]:
zones_da = xr.DataArray(
        data=zones,
        dims=['y', 'x'],
        coords={
            'y': raster.y,
            'x': raster.x
        }
    )# .chunk(raster.chunksizes)
zones_da

<xarray.DataArray (y: 73988, x: 30913)> Size: 9GB
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])
Coordinates:
  * y        (y) float64 592kB 3.151e+06 3.151e+06 ... 2.411e+06 2.411e+06
  * x        (x) float64 247kB -2.306e+06 -2.306e+06 ... -1.997e+06 -1.997e+06

In [8]:
stats_df = stats(
        zones=zones_da,
        values=raster,
        stats_funcs=['mean', 'sum', 'min', 'max', 'count'],
        nodata_values=raster.rio.nodata
    )
stats_df

,zone,mean,sum,min,max,count
0,0,299.271186,17657.0,2.0,1611.0,59.0
1,1,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
956475,967480,NaN,NaN,NaN,NaN,NaN
956476,967481,NaN,NaN,NaN,NaN,NaN
956477,967482,NaN,NaN,NaN,NaN,NaN
956478,967483,NaN,NaN,NaN,NaN,NaN


In [ ]:
stats_df.to_csv(f'high_res_zonal_stats_dask_{layer}_{vpu_id}.csv')